## MM1 model

In [1]:
import numpy as np
import pandas as pd

## 0) config

In [301]:
date = '2018-11-09_13h02'
base_path = '/Users/florianmorath/Desktop/ASL_project/scripts/3_ThroughputForWrites/processed_data/' + date + '/'
worker = 16
client = 144

## 1) fetch input parameters (service and arrival rate)

For the mean service rate \mu we just take the max observed throughput for each worker config

In [302]:
df_mw_tp_max = pd.read_csv('/Users/florianmorath/Desktop/ASL_project/scripts/6_QueuingModel/processed_data/' + date + '/' + 'full_write_mw_tp_max.csv')
df_mw_tp_max

,client,worker,write_tp_mean,write_tp_std
0,6,8,3304.442857,46.921663
1,6,16,3316.414286,40.795229
2,6,32,3313.628571,36.097170
3,6,64,3349.214286,28.547781
4,24,8,8446.400000,100.641007
5,24,16,8630.285714,98.976829
6,24,32,8470.171429,70.004335
7,24,64,8414.957143,66.697358
8,48,8,9093.485714,46.574767
9,48,16,10452.014286,713.507250


In [303]:
servicerate_mu = np.max(df_mw_tp_max[(df_mw_tp_max['worker']==worker)]['write_tp_mean'])
round(servicerate_mu, 2)

10891.57

In [304]:
# compute it from service time
#df_breakdown = pd.read_csv(base_path + 'full_write_mw_rt_breakdown.csv')
#df_cut = df_breakdown[(df_breakdown['client']==client)&(df_breakdown['worker']==worker)]
#measured_servicetime = df_cut['write_workerPreTime'].item() + df_cut['write_memcachedRTT'].item() + df_cut['write_workerPostTime'].item()
#servicerate_mu = 1/(measured_servicetime/1000)*worker*2
#servicerate_mu

In [305]:
df_mw_tp = pd.read_csv(base_path + 'full_write_mw_tp.csv')
df_mw_tp

,client,worker,write_tp_mean,write_tp_std
0,6,8,3250.019048,46.921663
1,6,16,3264.114286,40.795229
2,6,32,3270.285714,36.097170
3,6,64,3327.504762,28.547781
4,24,8,8304.466667,100.641007
5,24,16,8498.238095,98.976829
6,24,32,8414.190476,70.004335
7,24,64,8335.228571,66.697358
8,48,8,9037.752381,46.574767
9,48,16,9920.252381,713.507250


For the mean arrival rate \lambda we take the throughput at the specific configuration

In [306]:
arrivalrate_lambda = df_mw_tp[(df_mw_tp['worker']==worker) & (df_mw_tp['client']==client)]['write_tp_mean'].item()
round(arrivalrate_lambda, 2)

10716.8

## 2) apply formulas (make predictions)

In [307]:
trafficload_rho = arrivalrate_lambda / servicerate_mu
round(trafficload_rho, 4)

0.984

number of jobs:

In [308]:
mean_number_jobs_queue = trafficload_rho**2/(1-trafficload_rho)
round(mean_number_jobs_queue, 2)

60.33

In [309]:
mean_number_jobs_service = trafficload_rho
round(trafficload_rho, 2)

0.98

In [310]:
mean_number_jobs_system =  trafficload_rho/(1-trafficload_rho)
round(mean_number_jobs_system, 2)

61.32

response times:

In [311]:
mean_waittime = 1000*trafficload_rho*(1/servicerate_mu)/(1-trafficload_rho)
round(mean_waittime, 2) # in ms 

5.63

In [312]:
mean_servicetime = 1000*1/servicerate_mu
round(mean_servicetime, 2)

0.09

In [313]:
mean_total_time = 1000*(1/servicerate_mu)/(1-trafficload_rho)
round(mean_total_time, 2)

5.72

## 3) fetch measured values

number of jobs:

In [314]:
# queue length
df_ql = pd.read_csv(base_path + 'full_write_mw_queuelength.csv')
measured_number_jobs_queue = df_ql[(df_ql['client']==client)&(df_ql['worker']==worker)]['write_queueLength'].item()
round(measured_number_jobs_queue, 2)

48.39

In [315]:
# number of jobs in service
measured_number_jobs_service = worker*2*trafficload_rho
round(measured_number_jobs_service, 2)

31.49

In [316]:
# number of jobs in the system
measured_number_jobs_system = measured_number_jobs_queue + measured_number_jobs_service
round(measured_number_jobs_system, 2)

79.88

response times:

In [317]:
# service time in ms
df_breakdown = pd.read_csv(base_path + 'full_write_mw_rt_breakdown.csv')
df_cut = df_breakdown[(df_breakdown['client']==client)&(df_breakdown['worker']==worker)]
measured_servicetime = df_cut['write_workerPreTime'].item() + df_cut['write_memcachedRTT'].item() + df_cut['write_workerPostTime'].item()
round(measured_servicetime, 2)



2.99

In [318]:
# total response time
df_mw_rt = pd.read_csv(base_path + 'full_write_mw_rt.csv')
measured_totaltime = df_mw_rt[(df_breakdown['client']==client)&(df_breakdown['worker']==worker)]['write_rt_mean'].item()
round(measured_totaltime, 2)



12.77

In [319]:
# queue time
measured_waittime = measured_totaltime - measured_servicetime
round(measured_waittime, 2)

9.78